In [1]:
import pennylane as qml
import pennylane.numpy as np

In [2]:
n_qubits = 9
dev = qml.device("default.qubit", wires=n_qubits)
error_dict = {0: 'PauliX', 1: 'PauliY', 2: 'PauliZ'}

def error(error_key, qubit):
    """Defines the error that is induced in the circuit.

    Args:
        error_key (int): An integer associated to the type of error (Pauli X, Y, or Z)
        qubit (int): The qubit that the error occurs on.
    """
    getattr(qml, error_dict[error_key])(qubit)

@qml.qnode(dev)
def shor(state, error_key, qubit):
    """A circuit defining Shor's code for error correction.

    Args:
        state (list(float)): The quantum state of the first qubit in the circuit.
        error_key (int): An integer associated to the type of error (Pauli X, Y, or Z)
        qubit (int): The qubit that the error occurs on.

    Returns:
        (list(float)): The expectation value of the Pauli Z operator on every qubit.
    """
    qml.QubitStateVector(np.array(state), wires=0)
        # Put your code here #
    for i in dev.wires[::3][1:]:
        qml.CNOT(wires=[0,i])
    for i in dev.wires[::3]:
        qml.Hadamard(wires=i)
        qml.CNOT(wires=[i,i+1])
        qml.CNOT(wires=[i,i+2])
        
    error(error_key, qubit)
    
    for i in dev.wires[::3]:
        qml.CNOT(wires=[i,i+1])
        qml.CNOT(wires=[i,i+2])
        qml.Toffoli(wires=[i+2,i+1,i])
        qml.Hadamard(wires=i)
    for i in dev.wires[::3][1:]:
        qml.CNOT(wires=[0,i])
    qml.MultiControlledX(wires=[*dev.wires[::3][1:], 0])

    return [qml.expval(qml.PauliZ(i)) for i in dev.wires]

In [4]:
shor([0, 1], 0, 3)

[tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(1., requires_grad=True)]